# Vectors

We will begin by making an abstract vector class

In [87]:
from abc import ABC, abstractmethod

class vector(ABC):
  """An abstract class for vectors for inverse problems"""
  def __init__(self):
    """Default initializer for an abstract class"""
    pass;

  @abstractmethod
  def get_nd_array(self):
    """Return numpy array representation of the vector"""


  @abstractmethod
  def check_same(self,vec2):
    """Check to see if another vector belongs to the
       same vector space
    """

  @abstractmethod
  def scale(self,sc:float):
    """Scale a vector by another vector"""

  @abstractmethod
  def scale_add(self,vec2,sc1:float=1,sc2:float=1):
    """Scale a vector by another vector, scaling both

        self*sc1+sc2*vec2

        vec2 - Vector to add to the current vector
        sc1 -  How much to scale the current vector by
        sc2 -  How much to scale the vec2 by


    """

  @abstractmethod
  def dot(self,vec2):
    """Calculate the dot product of a vector with the current vector

      vec2 - the vector to dot the current vector with

    """

  @abstractmethod
  def clone(self,spaceOnly=False):
    """Make a copy of the vector

        spaceOnly - Only clone the space

    """



In [88]:
import numpy as np
import copy
class vec_numpy(vector):

  def __init__(self,vals):
    if  isinstance(vals,np.ndarray):
      self._vals=copy.deepcopy(vals)
    if vals is None:
      self._vals=None

  def check_alloc(self,throwError=True):
    """Check to make sure vector is allocated"""
    if self._vals is None:
      if throwError:
        raise Exception("Vector not allocated")
      return False
    return True
  def get_nd_array(self):
    """Return nd_array representation"""
    self.check_alloc()
    return self._vals

  def scale(self,sc:float):
    """Scale a vector by a scalar float"""
    self.check_alloc()
    self._vals*=sc
    return self

  def scale_add(self,vec2,sc1:float=1,sc2:float=1):
    """Scale a vector by another vector, scaling both

        self*sc1+sc2*vec2

        vec2 - Vector to add to the current vector
        sc1 -  How much to scale the current vector by
        sc2 -  How much to scale the vec2 by


    """
    self.check_alloc()
    self._vals=self._vals*sc1+vec2.get_nd_array()*sc2
    return self

  def __str__(self):
    return f"shape={self._vals.shape}\n{self._vals}\n"
  def dot(self,vec2):
    """Calculate the dot product of a vector with the current vector

      vec2 - the vector to dot the current vector with

    """
    self.check_alloc()
    return np.dot(self.get_nd_array(),vec2.get_nd_array())

In [89]:
class vec_1d(vec_numpy):
  def __init__(self,n,o,d,vals=None,spaceOnly=False):
    self._n=n
    self._o=o
    self._d=d
    if self._d==0:
      raise Exception("d can't be 0")
    if vals is not None:
      if not isinstance(vals,np.ndarray):
        raise Exception("Expecting vals to be a numpy array")
      if  vals.dtype != np.float32 and vals.dtype!=np.float64:
        raise Exception(f"Expecting a float array {vals.dtype}")
      if len(vals.shape)!=1:
        raise Exception("Expecting a 1-D array")
      if vals.shape[0]!=self._n:
        raise Exception(f"Expecting the size of arr to be = {n}")
      super().__init__(vals)
    else:
      if spaceOnly:
        super().__init(None)
      else:
        super().__init__(np.zeros((self._n,),dtype=np.float32))


  def check_same(self,vec2):
    if not isinstance(vec2,vec_1d):
      return False
    if vec2._n != self._n:
      return False

    if (self._o-vec2._o)/self._d > .001:
      return False
    if (self._d-vec2._d)/self._d > .001:
      return False
    return True

  def clone(self,spaceOnly=False):
    """Clone a vector, potentionally spaceOnly

      spaceOnly=False

    """
    return vec_1d(self._n,self._o,self._d,self._vals,spaceOnly)



In [90]:
class operator(ABC):
  """An abstract class for operators for inverse problems"""
  def __init__(self,domain,range):
    """Default initializer for an abstract class

      domain - Domain of operator
      range. - Range of operator

    """
    self._domain=domain.clone(spaceOnly=True)
    self._range=range.clone(spaceOnly=True)

  @abstractmethod
  def forward(self,add,model,data):
    """
      Run a forward

      add - Whether or not to add to the output
      model - Model (input)
      data. - Data (output)


    """


  @abstractmethod
  def adjoint(self,add,model,data):
    """
      Run an adjoint

      add - Whether or not to add to the output
      model - Model (output)
      data. - Data (output)


    """

  def check_same(self,model,data):
    """
      Check to see if model and data match operator initialization

        model - Model space
        data. - data space
    """
    if not self._domain.check_same(model):
      raise Exception("model and domain don't match")

    if not self._range.check_same(data):
      raise Exception("range and data don't match")


In [91]:
class matrix_mult(operator):
  """Perform matrix multiplication"""
  def __init__(self,mod,dat,mat):
    """
      Initialize matrix multiplication operator
      mod - Domain
      dat - Range
      mat - Matrix to multiply

    """
    if not isinstance(mod,vec_1d):
      raise Exception("Expecting model to be vec_1d")
    if not isinstance(dat,vec_1d):
      raise Exception("Expecting data to be vec_1d")
    if not isinstance(mat,np.ndarray):
      raise Excception("Expecting mat to be a 1-D array")
    if mod._n != mat.shape[1] or dat._n!=mat.shape[0]:
      raise Exception("Matrix doesn't match model and data")
    super().__init__(mod,dat)

  def forward(self,add,model,data):
    """
      Run a forward

      add - Whether or not to add to the output
      model - Model (input)
      data. - Data (output)


    """
    self.check_same(model,data)
    if not add: data.scale(0.)

    d=data.get_nd_array()
    m=model.get_nd_array()

    for j in range(d.shape[0]):
      for i in range(m.shape[0]):
        d[j]+=mat[j,i]*m[i]

  def adjoint(self,add,model,data):
    """
      Run an adjoint

      add - Whether or not to add to the output
      model - Model (output)
      data. - Data (output)


    """
    self.check_same(model,data)
    if not add: model.scale(0.)

    d=data.get_nd_array()
    m=model.get_nd_array()

    for j in range(d.shape[0]):
      for i in range(m.shape[0]):
        m[i]+=mat[j,i]*d[j]



In [97]:
mat=np.array([[1.,2.],[7.,-4.]])
mod=vec_1d(2,0.,1.)
dat=vec_1d(2,0.,1.)
mat_op=matrix_mult(mod,dat,mat)
mod.get_nd_array()[:]=[-2,4]
mat_op.forward(False,mod,dat)
m2=mod.clone()
mat_op.adjoint(False,m2,dat)


SyntaxError: ignored